In [2]:
import pandas as pd
import geopandas as gpd
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium

C:\Users\gabri\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Centroid

In [15]:
y = region_map.centroid.y.iloc[0]
y

C:\Users\gabri\AppData\Local\Temp\ipykernel_6560\2836198186.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = region_map.centroid.y.iloc[0]


-22.72906184797807

In [16]:
x = region_map.centroid.x.iloc[0]
x

C:\Users\gabri\AppData\Local\Temp\ipykernel_6560\566618580.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = region_map.centroid.x.iloc[0]


-43.37687006952975

## HeatMap

In [196]:
from folium.plugins import HeatMap

base = folium.Map([y, x], zoom_start = 11, tiles = 'OpenStreetMap')

HeatMap(data, name="Origin of the trips").add_to(base)

folium.LayerControl().add_to(base)

base.save('HeatMap.html')

In [197]:
match_day

,origin_id,destination_id,total_trips
0,0,182,2733.67
1,1,182,20.38
2,2,182,9404.40
3,3,182,142.31
4,4,182,613.65
5,5,182,238.29
6,6,182,723.79
7,7,182,873.64
8,8,182,88.61
9,9,182,425.03


## HeatMap

In [7]:
match_day = pd.read_csv('../../data/trips_reports/my_report_matchday(20-4).csv') #from claro-br.kidodynamics
region_map = gpd.read_file('../../data/RIO DE JANEIRO MAPS/final_map/final_map2.geojson')

In [8]:
#Function to normalize the "total_trips" column
#total_trips is str and has some '<10' values
def normalize(df):
    for key, value in df["total_trips"].iteritems():
        if value == '<10':
            df.at[key, 'total_trips'] = '5'
    
    df['total_trips'] = df['total_trips'].astype(float)
    print(df['total_trips'].sum())

In [9]:
normalize(match_day)

585672.79


C:\Users\gabri\AppData\Local\Temp\ipykernel_6560\3283907314.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in df["total_trips"].iteritems():


In [10]:
#merging match_day with region_map
match_region = pd.merge(region_map, match_day, how='left', left_on=['ID'], right_on=['origin_id'])

In [11]:
#removing stadium as a origin rows
match_region = match_region[:183]

In [12]:
#removing trips inside stadium area
match_region.at[182, 'total_trips'] = 0

In [13]:
#filling the NaN spaces 
match_region.fillna(0, inplace=True)

In [14]:
match_region.head(5)

,ID,name,poly_type,geometry,origin_id,destination_id,total_trips
0,0,BELFORD ROXO,periphery,"POLYGON ((-43.37030 -22.69426, -43.37009 -22.6...",0.0,182.0,2733.67
1,1,CACHOEIRAS DE MACACU,periphery,"POLYGON ((-42.79038 -22.37813, -42.78943 -22.3...",1.0,182.0,20.38
2,2,DUQUE DE CAXIAS,periphery,"POLYGON ((-43.41519 -22.60109, -43.41566 -22.6...",2.0,182.0,9404.40
3,3,GUAPIMIRIM,periphery,"POLYGON ((-42.95809 -22.46615, -42.95703 -22.4...",3.0,182.0,142.31
4,4,ITABORAÍ,periphery,"POLYGON ((-42.94340 -22.70671, -42.94301 -22.7...",4.0,182.0,613.65


In [23]:
data = []
for i in range(len(match_region)):
    data.append([match_region['geometry'][i].centroid.y,
                 match_region['geometry'][i].centroid.x,
                 match_region['total_trips'][i]])

#data = [[lon1, lat1, total_trips1],[lon2, lat2, total_trips2], ...]

### Creating the Heat Map using Folium plugin

In [24]:
from folium.plugins import HeatMap

base = folium.Map([y, x], zoom_start = 11, tiles = 'OpenStreetMap')

HeatMap(data, name="Origin of the trips").add_to(base)

folium.LayerControl().add_to(base)

base.save('HeatMap.html')